In [23]:
import numpy as np
import pandas as pd

In [24]:
data = pd.read_csv('data_train_test.csv')
a = data['patient_manual_class']
a = a.replace(np.nan, 0)
a = a.replace(2, 0)
data['patient_manual_class'] = a
data = data.dropna()

In [25]:
from sklearn.model_selection import train_test_split
num_features = ['abs', 'std', 'norm_abs', 'mean', 'sd', 'median', 'mode', 'Q25', 'Q75', 'IQR', 'skew', 'kurt', 'kx2', 'n_pic']
cat_features = ['patient_ds', 'rec_date_time']
X_train, X_test, y_train, y_test = train_test_split(data[num_features], data['patient_manual_class'], test_size=0.2, shuffle=False)

In [26]:
df = pd.DataFrame(data[num_features])

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold 
#пытаемся улучшить долю правильных ответов, находим оптимальное кол-во деревьев в модели случайного леса
# Инициализируем валидацию
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
trees_grid = [5, 10, 15, 20, 30, 50, 75, 100]
# Обучаем на тренировочном датасете
for ntrees in trees_grid:
    model_RF = RandomForestClassifier(n_estimators=ntrees, random_state=42, n_jobs=-1, oob_score=True)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(X_train, y_train):
        model_RF.fit(X_train, y_train)
        temp_train_acc.append(model_RF.score(X_train, y_train))
        temp_test_acc.append(model_RF.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Наибольшая точность модели случайного леса определяется в {:.2f}% на {} деревьях".format(max(test_acc.mean(axis=1))*100, 
                                                        trees_grid[np.argmax(test_acc.mean(axis=1))]))

C:\Users\Chernyavskaya_OS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
C:\Users\Chernyavskaya_OS\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_forest.py:586: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


Наибольшая точность модели случайного леса определяется в 86.67% на 15 деревьях


In [28]:
#пытаемся улучшить долю правильных ответов, находим оптимальное кол-во деревьев в модели бэггинга на деревьях решений
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
# Инициализируем валидацию
skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
trees_grid = [5, 10, 15, 20, 30, 50, 75, 100]
# Обучаем на тренировочном датасете
for ntrees in trees_grid:
    model_BG_DT = BaggingClassifier(DecisionTreeClassifier(),n_estimators=ntrees, random_state=42)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(X_train, y_train):
        model_BG_DT.fit(X_train, y_train)
        temp_train_acc.append(model_BG_DT.score(X_train, y_train))
        temp_test_acc.append(model_BG_DT.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Наибольшая точность модели бэггинга на дереве решений определяется в {:.2f}% на {} деревьях".format(max(test_acc.mean(axis=1))*100, 
                                                        trees_grid[np.argmax(test_acc.mean(axis=1))]))

Наибольшая точность модели бэггинга на дереве решений определяется в 83.33% на 10 деревьях


In [29]:
from sklearn.svm import SVC
model_SVM = SVC(gamma='auto').fit(X_train, y_train)
prediction = model_SVM.predict(X_test)
from sklearn import metrics
print("Точность:", metrics.accuracy_score(y_test, prediction))

Точность: 0.8


In [30]:
from sklearn.svm import SVC
model_SVM = SVC(kernel='linear').fit(X_train, y_train)
prediction = model_SVM.predict(X_test)
from sklearn import metrics
print("Точность:", metrics.accuracy_score(y_test, prediction))

Точность: 0.8


In [31]:
#подбор ядра и ошибки штрафа для метода опорных векторов
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn import metrics

models_SVC = []
models_SVC.append(('Метод опорных векторов на ядре rbf, ошибка штрафа С=1000000', SVC(kernel='rbf', C=1000000)))
models_SVC.append(('Метод опорных векторов на ядре rbf, ошибка штрафа С=.0000001', SVC(kernel='rbf', C=.0000001)))
models_SVC.append(('Метод опорных векторов на ядре радиальной базисной функции:', SVC(kernel='rbf')))
models_SVC.append(('Метод опорных векторов на полиномиальном ядре:', SVC(kernel='poly')))
models_SVC.append(('Метод опорных векторов на сигмоидальном ядре:', SVC(kernel='sigmoid')))
results = []
names = []
for name, model in models_SVC:
    kfold = StratifiedKFold(n_splits=2, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std())) 

Метод опорных векторов на ядре rbf, ошибка штрафа С=1000000: 0.691667 (0.002778)
Метод опорных векторов на ядре rbf, ошибка штрафа С=.0000001: 0.788889 (0.000000)
Метод опорных векторов на ядре радиальной базисной функции:: 0.788889 (0.000000)
Метод опорных векторов на полиномиальном ядре:: 0.788889 (0.000000)
Метод опорных векторов на сигмоидальном ядре:: 0.775000 (0.013889)


In [32]:
#Cтроим модель по оптимальным параметрам и получаем результаты классификации в файле .csv
model_RF = RandomForestClassifier(n_estimators=15, random_state=42, n_jobs=-1, oob_score=True).fit(X_train, y_train)
data_test = pd.read_csv('data_test.csv')
df_test = pd.DataFrame(data_test[num_features])
df_test = df_test.dropna()
X_test = df_test.iloc[:,:]
prediction = model_RF.predict(X_test)
df_test["fistulas_kind"] = prediction #добавляем в наш НД расчетное значение
#пишем в файл результат, колонка fistulas_kind:
df_test.to_csv("data_test_model_RF_15.csv", index=False)
print("Результаты классификации модели, построенной по методу случайного леса с оптимальными параметрами выгружены в файл data_test_model_RF_15.csv")

Результаты классификации модели, построенной по методу случайного леса с оптимальными параметрами выгружены в файл data_test_model_RF_15.csv


In [33]:
import pickle
data = model_RF
with open('model_RF_15.pickle', 'wb') as f:
  pickle.dump(data, f)

In [34]:
print("Виды АВФ в результате классификации методом случайного леса: ", df_test.fistulas_kind.unique())
print("0 - плохая, 1 - хорошая")
print("\n", df_test.groupby('fistulas_kind').size())

Виды АВФ в результате классификации методом случайного леса:  [1. 0.]
0 - плохая, 1 - хорошая

 fistulas_kind
0.0     54
1.0    180
dtype: int64


In [35]:
print("Демонстрация окончена")

Демонстрация окончена
